In [29]:
import pandas as pd
import numpy as np
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error


# seaborn can be used to "prettify" default matplotlib plots by importing and setting as default
import seaborn as sns
sns.set() # Set searborn as default

In [30]:
data = pd.read_csv('case1Data.csv')
y = data['y']
X = data.drop('y', axis=1)
# Get number of observations (n) and number of independent variables (p)
[n, p] = np.shape(X)

In [31]:
# impute missing values with mean
X_imputed_mean = X.fillna(X.mean())

# check if there are any columns with zero variance
variance = np.var(X_imputed_mean, axis=0)
zero_var_cols = np.where(variance == 0)[0]
print('Zero variance columns:', zero_var_cols)

# drop columns with zero variance
X_imputed_mean = X_imputed_mean.drop(X_imputed_mean.columns[zero_var_cols], axis=1)
print('Columns dropped:', X.columns[zero_var_cols])

# devide the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_imputed_mean, y, test_size=0.2, random_state=42)

# standardize the data
scaler = StandardScaler()
X_train_standardardized = scaler.fit_transform(X_train)
X_test_standardardized = scaler.transform(X_test)

X_standardized = scaler.fit_transform(X_imputed_mean)

Zero variance columns: [96]
Columns dropped: Index(['C_02'], dtype='object')


In [ ]:
lasso = Lasso()
param_grid = {'alpha': np.logspace(-4, 4, 100)}
grid_search = GridSearchCV(lasso, param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_standardized, y)
best_lasso = grid_search.best_estimator_

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.460e+01, tolerance: 4.391e+01
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.260e+01, tolerance: 4.239e+01
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iter

In [33]:
rmse = np.sqrt(-cross_val_score(best_lasso, X_standardized, y, cv=5, scoring='neg_mean_squared_error'))
print('best alpha:', best_lasso.alpha)
print('Mean RMSE:', np.mean(rmse))
print('Standard deviation of RMSE:', np.std(rmse))

best alpha: 0.6280291441834259
Mean RMSE: 30.372947039542918
Standard deviation of RMSE: 5.218510326217925


# Lasso on principal components

In [51]:
pca_data = pd.read_csv('pca_data.csv')
X = pca_data.drop('y', axis=1)
# take the first 40 principal components
X_40 = X.iloc[:, :40]

In [52]:
lasso = Lasso()
param_grid = {'alpha': np.logspace(-4, 4, 100)}
grid_search = GridSearchCV(lasso, param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_40, y)
best_lasso = grid_search.best_estimator_

In [53]:
rmse = np.sqrt(-cross_val_score(best_lasso, X_40, y, cv=5, scoring='neg_mean_squared_error'))
print('best alpha:', best_lasso.alpha)
print('Mean RMSE:', np.mean(rmse))
print('Standard deviation of RMSE:', np.std(rmse))

best alpha: 1.592282793341094
Mean RMSE: 40.23678726305708
Standard deviation of RMSE: 7.519826464963874
